In [ ]:
import numpy as np
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
from keras.applications.nasnet import NASNetMobile
from keras.applications.mobilenet import MobileNet
from keras.utils import np_utils
import matplotlib.pyplot as plt
from pandas import DataFrame

%matplotlib inline

## Load Data

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
print(y_train[7])   #Convince yourself that the labels are consistent with the data.  See https://www.cs.toronto.edu/~kriz/cifar.html
plt.imshow(x_train[7,:,:,:])

## Clean Data

In [ ]:
x_train_unrolled = x_train.reshape(-1,32*32*3) / 256
print(x_train_unrolled.shape)
y_train_encoded = np_utils.to_categorical(y_train)

x_test_unrolled = x_test.reshape(-1,32*32*3) / 256
y_test_encoded = np_utils.to_categorical(y_test)

## Load Existing Neural Net

In [ ]:
fresh_layers = fresh.layers
fresh_layers
MobileNet??

In [ ]:
fresh = MobileNet(include_top = False, weights=None)
#fresh.summary()

In [ ]:
rand_init_model = Sequential([
    Dense(32,input_shape=(32,32,3), activation = 'sigmoid',name='Named'),
    fresh,
    Dense(10,activation = 'softmax')
])
rand_init_model.add(fresh.get_layer())

In [ ]:
%%time
#Documentation for MobileNet says smaller shapes allowed, but throws error.  https://keras.io/applications/#mobilenet
#Thus, we have to adapt a new model to take the dimensions of our data (32,32,3).
#Initializing w/ input_shape will create input layer which we will later remove.  Hacky.
trained_model = MobileNet(input_shape =(128,128,3), include_top = False, weights='imagenet') 

#Create new model to add the trained model into.  
new_model = Sequential()
new_model.add(Dense(32,input_dim=(32*32*3), activation = 'sigmoid',name='Named'))
    
    
#Extend list of layers to include layers of trained_model, except for the first input layer.
#Check summary on model to see the layer structures.
#Remember, include_top = False has already chopped off the classification layers.
new_model.layers.extend(trained_model.layers[1:])
new_model.add(Dense(10, activation = 'softmax'))

#Experiment: freeze all trainable params.  What do you expect to happen to the acc?  Check summary.
#new_model.trainable = False  


new_model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
history_new = new_model.fit(x_train_unrolled,y_train_encoded,epochs=5,batch_size=4)


In [ ]:
new_model.summary()

In [ ]:
plt.plot(history.history['loss'])